# Wuhan Outbound Flights
data source: history.adsbexchange.com
- additional data preprocessing was done before deriving at 'wuhan_int_airport_outbound_Jan2020.csv'
- each record(row) in the data represents a single receiver flight message from the airplane

Timeline: all flights on **2nd Jan 2020** outbound of Wuhan Tianhe Airport

purpose: to track and simulate the number of infected people in Singapore

In [1]:
from glob import glob
import os
import time
import re
import numpy as np
import pandas as pd
import pickle as pkl
import math
import json
import warnings
pd.set_option('display.max_columns',300)
pd.set_option('display.max_rows',300)
warnings.filterwarnings('ignore')

In [2]:
zhhh = pd.read_csv('./Data/wuhan_int_airport_outbound_Jan2020.csv')

In [3]:
def getToCountry(row):
    out = row.split(', ')
    return out[-1]

In [4]:
zhhh['To_country'] = zhhh['To'].apply(getToCountry)

### Top Outbound country from Wuhan Tianhe airport on 2 Jan 2020

In [5]:
zhhh.groupby(['From','To_country']).agg({'Id':'nunique'}).sort_values(by='Id', ascending = False)

Id
From                     To_country        
ZHHH Wuhan Tianhe, China China           51
                         Thailand         6
                         Japan            4
                         Viet Nam         3
                         Macau            2
                         Malaysia         2
                         South Korea      2
                         Hong Kong        1
                         Singapore        1
                         Taiwan           1
                         Turkey           1
                         United Kingdom   1
                         United States    1

### Top Domestic Outbound City on 2 Jan 2020

In [6]:
zhhh[zhhh['To_country']=='China'].groupby(['From','To']).agg({'Id':'nunique'}).sort_values(by='Id', ascending = False)

Id
From                     To                                
ZHHH Wuhan Tianhe, China ZSPD Shanghai Pudong, China     12
                         ZBAA Beijing Capital, China     11
                         ZSSS Shanghai Hongqiao, China    6
                         ZJHK Haikou Meilan, China        5
                         ZUUU Chengdu Shuangliu, China    3
                         ZYCC Longjia, Changchun, China   3
                         ZBLA Dongshan, Hailar, China     2
                         ZGGG Guangzhou Baiyun, China     2
                         ZGSZ Shenzhen Bao'an, China      2
                         ZSHC Hangzhou Xiaoshan, China    2
                         ZUCK Chongqing Jiangbei, China   2
                         ZYHB Taiping, Harbin, China      2
                         ZYTX Taoxian, Shenyang, China    2
                         ZPPP Kunming Wujiaba, China      1
                         ZSNB Ningbo Lishe, China         1

### Operator and Model of airplanes to Singapore

In [7]:
zhhh[zhhh['To']=='WSSS Singapore Changi, Singapore'].groupby('Op').agg({'Id':'nunique'}).sort_values(by='Id', ascending = False)

,Id
Op,
Scoot,1


In [8]:
zhhh[zhhh['To']=='WSSS Singapore Changi, Singapore'].groupby('Call').agg({'Id':'nunique'}).sort_values(by='Id', ascending = False)

,Id
Call,
TGW121,1


In [9]:
zhhh[zhhh['To']=='WSSS Singapore Changi, Singapore'].groupby('Mdl').agg({'Id':'nunique'}).sort_values(by='Id', ascending = False)

,Id
Mdl,
Boeing 787 9,1


## Scoot flights from Wuhan (Direct flight) to Singapore and vice versa on 2 Jan 2020
- TR120 and TR121 services WUH - SIN route
- flight service was cancelled officially between 
- TR120: 23 January 2020 – 2 February 2020 (SIN - WUH)
- TR121: 24 January 2020 – 3 February 2020 (WUH - SIN)

-- TR121 (or TGW121 in our data) is the Inbound flight from Wuhan to SG

-- Assuming the flight standard schedule of one flight per day$^{1}$ by Scoot TR121, from 2 Jan (date which SG MOH issued precautionary measures on Wuhan virus$^{2}$) to 23 Jan (Scoot's TR121's last trip$^{1}$), a total of **23** trips have been completed.

-- a Boeing 787-9 plane can have a passenger capacity of up to **280** + some crew members ~ **15**   $^{3}$

$^{1}$https://www.flyscoot.com/en/announcements/scoot-flights-to-and-from-wuhan

$^{2}$https://www.moh.gov.sg/news-highlights/details/precautionary-measures-in-response-to-severe-pneumonia-cases-in-wuhan-china

$^{3}$https://en.wikipedia.org/wiki/Boeing_787_Dreamliner#787-9

#### Estimated inbound passengers from WUH to SIN

In [10]:
(280+15)*(23-2+1)

6490

# Singapore Inbound Flights from affected Chinese states

#### List of domestic destinations of outbound flights from WUH airport
['ZBAA Beijing Capital, China',
 'ZBLA Dongshan, Hailar, China',
 'ZGGG Guangzhou Baiyun, China',
 "ZGSZ Shenzhen Bao'an, China",
 'ZJHK Haikou Meilan, China',
 'ZPPP Kunming Wujiaba, China',
 'ZSHC Hangzhou Xiaoshan, China',
 'ZSNB Ningbo Lishe, China',
 'ZSPD Shanghai Pudong, China',
 'ZSSS Shanghai Hongqiao, China',
 'ZUCK Chongqing Jiangbei, China',
 'ZUUU Chengdu Shuangliu, China',
 'ZYCC Longjia, Changchun, China',
 'ZYHB Taiping, Harbin, China',
 'ZYTX Taoxian, Shenyang, China']

In [12]:
sg_inb = pd.read_csv('./Data/chinese_to_singapore_2Jan2020.csv')

### Top Chinese domestic state with most number of flights to SG 

In [14]:
sg_inb.groupby(['From','To']).agg({'Id':'nunique'}).sort_values(by='Id', ascending = False)

,,Id
From,To,
"ZSPD Shanghai Pudong, China","WSSS Singapore Changi, Singapore",13
"ZBAA Beijing Capital, China","WSSS Singapore Changi, Singapore",5
"ZGGG Guangzhou Baiyun, China","WSSS Singapore Changi, Singapore",5
"ZUUU Chengdu Shuangliu, China","WSSS Singapore Changi, Singapore",4
"ZGSZ Shenzhen Bao'an, China","WSSS Singapore Changi, Singapore",3
"ZSHC Hangzhou Xiaoshan, China","WSSS Singapore Changi, Singapore",2
"ZUCK Chongqing Jiangbei, China","WSSS Singapore Changi, Singapore",2
"ZYCC Longjia, Changchun, China","WSSS Singapore Changi, Singapore",2
"ZYTX Taoxian, Shenyang, China","WSSS Singapore Changi, Singapore",2


### Operators of inbound flights to SG from Chinese domestic states

In [15]:
sg_inb.groupby('Op').agg({'Id':'nunique'}).sort_values(by='Id', ascending = False)

,Id
Op,
Singapore Airlines,8
China Eastern Airlines,6
SilkAir,5
Air China,4
Scoot,4
China Southern Airlines,3
China Cargo Airlines,1
Chongqing Airlines,1
Juneyao Airlines,1


### Flight numbers of Operators of inbound flights to SG from Chinese domestic states

In [16]:
sg_inb.groupby('Call').agg({'Id':'nunique'}).sort_values(by='Id', ascending = False)

,Id
Call,
CCA403,2
SIA825,2
CES545,2
SLK937,1
SIA827,1
SIA831,1
SIA833,1
SIA851,1
SLK935,1


### Model of airplanes of inbound flights to SG from Chinese domestic states

In [18]:
sg_inb.groupby('Mdl').agg({'Id':'nunique'}).sort_values(by='Id', ascending = False)

,Id
Mdl,
Airbus A320 214,5
Boeing 777 312ER,5
Airbus A330 243,4
Boeing 737NG 8SA/W,4
Boeing 787 10,2
Airbus A330 343E,2
Boeing 787 9,2
Airbus A320 251NSL,2
Airbus A321 253NSL,1
